In [ ]:
println(s"Current spark version is ${spark.version}")

In [ ]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

    
val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("tweet")

println(s"Total tweets in file: ${raw_sentiment.count}")


In [ ]:
import java.util.{Calendar, Timer, TimerTask}
//import java.nio.file.{Path, Paths, Files}

//Files.createTempDirectory("/home/jovyan/data/events-stream", null)

val timer = new Timer()

val task = new TimerTask {
  def run(): Unit = {
      val data = raw_sentiment.sample(fraction=0.00001,withReplacement=true)
      data.coalesce(1).write.format("json").mode("append").save("/home/jovyan/data/events-stream")
      println(s"${Calendar.getInstance().toInstant} - saved some data to the events stream!")
  } 
}

println("Streaming started!")

timer.schedule(task, 1000L, 1000L)

In [ ]:
task.cancel()